In [1]:
!pip install openpyxl

import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile
import pickle as pkl

def get_script(path, dic={}):
    for dir in listdir(path):
        if dir.startswith('TC'):
            for file in listdir(path+dir+'/'):
                dic[dir] = path+dir+'/'+file
        elif not isfile(path+dir+'/'):
            get_script(path+dir+'/', dic)
    return dic

test_cases_df = pd.read_excel('Katalon Capstone - Manual Test Cases.xlsx')

for column in test_cases_df.columns:
    if 'Unnamed: ' in column:
        test_cases_df.drop(column, inplace=True, axis=1)

test_cases_df.drop(test_cases_df.index[[0, 1]], inplace=True)

test_cases_df.reset_index(drop=True, inplace=True)

test_cases_df.iloc[131]['Title'] = 'TC132_Verify Question is timed out_P1'

path_dict = get_script('ubc-script-generation/ontest-main/Automation/Scripts/Main Test Cases/')

scripts = []
for i in range(test_cases_df.shape[0]):
    if test_cases_df.iloc[i]['Title'] in path_dict.keys():
        with open(path_dict[test_cases_df.iloc[i]['Title']]) as f:
            scripts.append(f.read())
    else:
        scripts.append(np.nan)

test_cases_df['test_scripts'] = scripts

In [2]:
test_cases_update = pd.read_csv('/Users/anshulkaushal/desktop/new_tc.csv')

In [3]:
test_cases_update

,Unnamed: 0,Feature,ID.No,Title,Test Scenario,Full manual test cases,TC Priority,Automated,New Test Cases,test_scripts
0,0,Login,TC001,TC001_Verify Successful Login With Valid Crede...,Verify login successfully with valid credentials,"Steps \n1. Navigate to OnTest site url ""https:...",P2,Yes,Steps \r1. Enter email address to Email textbo...,import static com.kms.katalon.core.checkpoint....
1,1,Login,TC002,TC002_Verify Failed Login With Invalid Credent...,Verify login successfully with valid credentials,Steps:\n1. Go to OnTest site (https://ontest.k...,P3,Yes,Steps:\r1. Enter email address to Email textbo...,import static com.kms.katalon.core.checkpoint....
2,2,Logout,TC003,TC003_Verify Logout System Successfully_P3,Verify login successfully with valid credentials,Pre-condition\nUser has logged in to OnTest su...,P3,Yes,"Steps:\r1. On the Dashboard page, click the av...",import static com.kms.katalon.core.testcase.Te...
3,3,Change Password,TC004,TC004_Verify Change Password Successfully_P2,Verify login successfully with valid credentials,Steps:\n1. Login to OnTest site (https://ontes...,P2,Yes,Steps:\r1. Click User's avatar on the top righ...,import static com.kms.katalon.core.testcase.Te...
4,4,Change Password,TC005,TC005_Verify Change Invalid Password_P4,Verify login successfully with valid credentials,Pre-condition\nUser has logged in to OnTest su...,P4,Yes,"Steps:\r1. On the Dashboard page, click the av...",import static com.kms.katalon.core.testcase.Te...
...,...,...,...,...,...,...,...,...,...,...
251,251,"Enhance Menu, Font, Tests listing page",TC252,TC252_Verify Navigation from Submitted on Ongo...,Login to the system and there is at least one ...,NaN,P2,Yes,NaN,NaN
252,252,"Enhance Menu, Font, Tests listing page",TC253,TC253_Verify Navigation from Reviewing on Ongo...,Login to the system and there is at least one ...,NaN,P2,Yes,NaN,NaN
253,253,"Enhance Menu, Font, Tests listing page",TC254,TC254_Verify Navigation from Invited on Comple...,Login to the system and there is at least one ...,NaN,P2,Yes,NaN,NaN
254,254,"Enhance Menu, Font, Tests listing page",TC255,TC255_Verify Navigation from Submitted on Comp...,Login to the system and there is at least one ...,NaN,P2,Yes,NaN,NaN


In [4]:
def segregate(case):
    """ (str) -> str, str, str
    Segregates a manual test case string into three sub-strings
    
    Params:
        case (str) a manual test case string
        
    Returns:
        pre_condition_str, steps_str, expected_result_str (str, str, str) 
        Three sub-strings corresponding to the pre-condition, steps and
        expected results section of a manual test case        
    """
    pre_condition = False
    steps = False
    expected_result = False

    pre_condition_str = ''
    steps_str = ''
    expected_result_str = ''
    
    for line in case.splitlines():
        if line.strip() == '':
            continue
        if 'pre-condition' in line.strip().lower():
            if len(line.split(':')) > 1 and not ((line.split(':')[1]).strip() == ''):
                pre_condition_str += line.split(':')[1]
            pre_condition = True
            continue
        if pre_condition or line.strip().lower().startswith('steps'):
            if line.strip().lower().startswith('steps'):
                pre_condition = False
                if len(line.split(':')) > 1 and not ((line.split(':')[1]).strip() == ''):
                    steps_str += line.split(':')[1]
                steps = True
                continue
            if pre_condition_str != '':
                pre_condition_str += '\n'
            pre_condition_str += line.rstrip()
        if steps or line.strip().lower().startswith('expected result'):
            if line.strip().lower().startswith('expected result'):
                steps = False
                if len(line.split(':')) > 1 and not ((line.split(':')[1]).strip() == ''):
                    expected_result_str += (line.split(':')[1])
                expected_result = True
                continue
            if steps_str != '':
                steps_str += '\n'
            steps_str += line.rstrip()
        if expected_result:
            if expected_result_str != '':
                expected_result_str += '\n'
            expected_result_str += line.rstrip()   
    
    return pre_condition_str, steps_str, expected_result_str

def process_test_cases(case_list):
    """ (list) -> list, list, list
    Processes list of manual test cases into three separate lists
    
    Params:
        case_list (list) a list of the manual test cases
        
    Returns:
        pre_condition_list, steps_list, expected_result_list (list, list, list) 
        Three lists of pre-condition, steps and expected results corresponding
        to each example in a list of the manual test cases 
    """
    pre_condition_list, steps_list, expected_result_list = [], [], []
    for case in case_list:
        if isinstance(case, float):
            pre_condition_list.append(np.nan)
            steps_list.append(np.nan)
            expected_result_list.append(np.nan)
        else:
            pre_condition, steps, expected_result = segregate(case)
            if pre_condition == '':
                pre_condition_list.append(np.nan)
            else:
                pre_condition_list.append(pre_condition)
            if expected_result == '':
                expected_result_list.append(np.nan)
            else:
                expected_result_list.append(expected_result)
            steps_list.append(steps)
    return pre_condition_list, steps_list, expected_result_list

test_cases_df['pre_conditions'], test_cases_df['steps'], test_cases_df['expected_results'] = process_test_cases(test_cases_df['Full manual test cases'].tolist())


In [5]:
test_cases_update['pre_condition'], test_cases_update['steps'], test_cases_update['expected_results'] = process_test_cases(test_cases_update['New Test Cases'].tolist())

In [6]:
import re

def process_steps(string):
    """ (str) -> str
    Strips the numerical numbering from the beginning of a string
    
    Params:
        string (str) a string
        
    Returns:
        (str)
        A string with the numerical numbering at its beginning removed
    """
    return re.sub('(\d+(\.\d+)?\.?)', '', string, count=1).strip()

def process(symbol, string):
    """ (str, str) -> str
    Strips the provided symbol from the beginning of a string
    
    Params:
        symbol (str) symbol to be stripped
        string (str) a string
    Returns:
        (str)
        A string with the symbol from its beginning removed
    """
    return string.replace(symbol, '', 1).strip()

In [7]:
def get_lists(column):
    """ (list) -> list
    Converts string blocks inside the list to nested lists of strings
    
    Params:
        column (list) a list of string blocks
    Returns:
        (list)
        A list containing nested lists of strings
    """
    lists = []
    for str_block in column:
        a_list = []
        if isinstance(str_block, float):
            lists.append(np.nan)
            continue
        else:
            a_nested_list = []
            another_nested_list = []
            double_nested = False
            if len(str_block.splitlines()) == 1:
                lists.append(str_block)
                continue
            for a_line in str_block.splitlines():
                if a_line.strip().startswith('-'):
                    double_nested = False
                    if len(a_nested_list) > 0:
                        a_list.append(a_nested_list.copy())
                    a_nested_list.clear()
                    a_list.append(process('-', a_line))
                elif a_line.strip().startswith('+'):
                    if double_nested:
                        if a_line.strip().startswith('++'):
                            another_nested_list.append(process('++', a_line))
                        else:
                            another_nested_list.append(process('+', a_line))
                    else:
                        if a_line.strip().startswith('++'):
                            a_nested_list.append(process('++', a_line))
                        else:
                            a_nested_list.append(process('+', a_line))
                elif a_line.strip().startswith('*'):
                    double_nested = True
                    if len(another_nested_list) > 0:
                        a_nested_list.append(another_nested_list.copy())
                    another_nested_list.clear()
                    another_nested_list.append(process('*', a_line))
            if double_nested:
                if len(another_nested_list) > 0:
                    a_nested_list.append(another_nested_list.copy())
            if len(a_nested_list) > 0:
                a_list.append(a_nested_list.copy())
        lists.append(a_list)
    return lists

pre_conditions_list = get_lists(test_cases_df['pre_conditions'].tolist())
expected_results_list = get_lists(test_cases_df['expected_results'].tolist())

In [8]:
steps_list = []
for steps_str in test_cases_df['steps'].tolist():
    steps = []
    if isinstance(steps_str, float):
        steps_list.append(np.nan)
    else:
        for step in steps_str.splitlines():
            steps.append(process_steps(step))
        steps_list.append(steps)

In [9]:
steps_list = []
for steps_str in test_cases_update['steps'].tolist():
    steps = []
    if isinstance(steps_str, float):
        steps_list.append(np.nan)
    else:
        for step in steps_str.splitlines():
            steps.append(process_steps(step))
        steps_list.append(steps)

In [10]:
with open('/Users/anshulkaushal/desktop/steps_updated.pkl', 'wb') as handle:
    pkl.dump(steps_list, handle)

In [11]:
# with open('/Users/anshulkaushal/desktop/steps.pkl', 'wb') as handle:
#     pkl.dump(steps_list, handle)
# with open('/Users/anshulkaushal/desktop/pre_conditions.pkl', 'wb') as handle:
#     pkl.dump(pre_conditions_list, handle)
# with open('/Users/anshulkaushal/desktop/expected_results.pkl', 'wb') as handle:
#     pkl.dump(expected_results_list, handle)

In [12]:
file_path = '/Users/anshulkaushal/desktop/funct-obj-dict.pkl'
func_obj_dict = pkl.load(open(file_path, "rb"))

In [13]:
print(func_obj_dict.keys())

dict_keys(['setText', 'click', 'waitForElementPresent', 'getText', 'verifyElementPresent', 'waitForElementNotPresent', 'verifyElementText', 'verifyElementVisible', 'waitForElementVisible', 'verifyElementNotPresent', 'verifyElementInViewport', 'findWebElements', 'findWebElement', 'sendKeys', 'selectOptionByValue', 'clearText', 'scrollToElement', 'getAttribute', 'verifyOptionSelectedByValue', 'verifyElementNotClickable', 'verifyElementNotChecked', 'waitForElementClickable'])


In [14]:
from collections import *

widget_dict_1 = defaultdict(lambda: defaultdict(set))
widget_dict_2 = dict()
for action in func_obj_dict.keys():
    for element in func_obj_dict[action]:
        list_1 = element.split('/')
        list_2 = list_1[-1].split('_')
        if len(list_2) == 1:
            list_2 = list_2[0].split('-')
        if len(list_1) == 2:
            widget_dict_1[list_1[0]][list_2[0]].add(element)
        elif len(list_1) == 3:
            try:
                widget_dict_2[list_1[0]][list_1[1]][list_2[0]].add(element)
            except:
                try:
                    widget_dict_2[list_1[0]][list_1[1]][list_2[0]] = set()
                    widget_dict_2[list_1[0]][list_1[1]][list_2[0]].add(element)
                except:
                    try:
                        widget_dict_2[list_1[0]][list_1[1]] = dict()
                        widget_dict_2[list_1[0]][list_1[1]][list_2[0]] = set()
                        widget_dict_2[list_1[0]][list_1[1]][list_2[0]].add(element)
                    except:
                        widget_dict_2[list_1[0]] = dict()
                        widget_dict_2[list_1[0]][list_1[1]] = dict()
                        widget_dict_2[list_1[0]][list_1[1]][list_2[0]] = set()
                        widget_dict_2[list_1[0]][list_1[1]][list_2[0]].add(element)
        else:
            print(element)

In [15]:
widget_dict = widget_dict_1.update(widget_dict_2)

In [16]:
widget_dict_1

defaultdict(<function __main__.<lambda>()>,
            {'Page_SearchTests': defaultdict(set,
                         {'txt': {'Page_SearchTests/txt_dropdown_menu',
                           'Page_SearchTests/txt_search',
                           'Page_SearchTests/txt_toggle_option'},
                          'btn': {'Page_SearchTests/btn_delete_dialog_cancel',
                           'Page_SearchTests/btn_delete_dialog_ok'},
                          'nav': {'Page_SearchTests/nav_testStatus'},
                          'cpn': {'Page_SearchTests/cpn_more',
                           'Page_SearchTests/cpn_test_modal',
                           'Page_SearchTests/cpn_tests'},
                          'toggle': {'Page_SearchTests/toggle_more'},
                          'successToast': {'Page_SearchTests/successToast'}}),
             'Page_Set Password': defaultdict(set,
                         {'txt': {'Page_Set Password/txt_Password',
                           'Page_Set Pass

In [17]:
for idx, (case, script) in enumerate(zip(test_cases_df['Full manual test cases'][0:8].tolist(), test_cases_df['test_scripts'][0:8].to_list())):
    print(idx)
    print()
    print(case)
    print()
    print(script)
    print('-----')
    print()

0

Steps 
1. Navigate to OnTest site url "https://ontest.katalon.com/"
2. Enter email address to Email textbox "admin1@mail.com"
3. Enter password to Password textbox "Admin@123"
4. Click button [Login]

Expected result
Dashboard page is loaded successfully

import static com.kms.katalon.core.checkpoint.CheckpointFactory.findCheckpoint
import static com.kms.katalon.core.testcase.TestCaseFactory.findTestCase
import static com.kms.katalon.core.testdata.TestDataFactory.findTestData
import static com.kms.katalon.core.testobject.ObjectRepository.findTestObject
import static com.kms.katalon.core.testobject.ObjectRepository.findWindowsObject
import com.kms.katalon.core.checkpoint.Checkpoint as Checkpoint
import com.kms.katalon.core.cucumber.keyword.CucumberBuiltinKeywords as CucumberKW
import com.kms.katalon.core.mobile.keyword.MobileBuiltInKeywords as Mobile
import com.kms.katalon.core.model.FailureHandling as FailureHandling
import com.kms.katalon.core.testcase.TestCase as TestCase
import c

In [18]:
func_obj_dict

{'setText': {'Page_Candidates/tbx_SearchCandidate',
  'Page_ChangePassword/tbx_ConfirmPassword',
  'Page_ChangePassword/tbx_CurrentPassword',
  'Page_ChangePassword/tbx_NewPassword',
  'Page_Create Member/inp_Email',
  'Page_Create Member/inp_Full Name',
  'Page_EmailTemplates/ipn_Subjective',
  'Page_Libraries/Library_Dialog/tbx_LibraryName',
  'Page_Libraries/Question_Dialog/tbx_ChoiceValue',
  'Page_Libraries/Question_Dialog/tbx_MaxTime',
  'Page_Libraries/Question_Dialog/tbx_Tags',
  'Page_Libraries/tbx_SearchQuestion',
  'Page_Login/tbx_Email',
  'Page_Login/tbx_Password',
  'Page_PreviewTest/textarea_Feedback',
  'Page_SearchTests/txt_search',
  'Page_SendFeedback/textarea_Feedback',
  'Page_Set Password/txt_Password',
  'Page_Set Password/txt_Repassword',
  'Page_TestCandidates/input_SearchCandidate',
  'Page_TestOverview/input_TestTags',
  'Page_TestOverview/input_TextOrNumberField',
  'Page_TestQuestions/tbx_AwardedScore',
  'Page_TestQuestions/tbx_MaxTime',
  'Page_TestQuesti

In [19]:
counter = 0
widgets_set = set()
for values in func_obj_dict.values():
    for value in values:
        counter += 1
        widgets_set.add(value.split('/')[-1].split('_')[0])
print(counter)

388


In [20]:
counter = 0
value_by_widget = defaultdict(set)
for element in widgets_set:
    print(element)
    print('*******')
    for values in func_obj_dict.values():
        for value in values:
            if value.split('/')[-1].split('_')[0] == element:
                counter += 1
                print(value)
                value_by_widget[element].add(value)
    print('*******')
print(counter)

btn-NewCandidate
*******
Page_Candidates/btn-NewCandidate
*******
switch
*******
Page_TestOverview/switch_TestAccessible
Page_TestOverview/switch_TestAccessible_Disable
*******
input
*******
Page_TestOverview/input_TestTags
Page_TestOverview/input_TextOrNumberField
Page_TestCandidates/input_SearchCandidate
Page_TestOverview/input_SwitchSetting
Page_TestOverview/input_RichText
Page_TestOverview/input_RichText
Page_TestOverview/input_TestTags
Page_TestOverview/input_TextOrNumberField
Page_TestOverview/input_SwitchSetting
*******
link
*******
Page_PreviewTest/link_PracticeTest
Page_Libraries/link_homeLib
Page_PreviewTest/link_PracticeTest
*******
successToast
*******
Page_AddTest/successToast
Page_AddTest/successToast
Page_SearchTests/successToast
Page_SearchTests/successToast
*******
Span
*******
Page_PreviewTest/Span_PreviewStatus
*******
dialog
*******
Page_TestQuestions/dialog_Dialog
Page_Libraries/Question_Dialog/dialog_UpdateQuestion
Page_TestQuestions/dialog_Dialog
Page_Libraries/d

In [21]:
widgets_set

{'Span',
 'btn',
 'btn-NewCandidate',
 'cbx',
 'ckedior',
 'ckeditor',
 'content',
 'cpn',
 'deleteOption',
 'dialog',
 'div',
 'icon',
 'img',
 'inp',
 'input',
 'ipn',
 'label',
 'lbl',
 'link',
 'modal',
 'nav',
 'radio',
 'rdi',
 'select',
 'slt',
 'span',
 'star',
 'successToast',
 'switch',
 'ta',
 'tab',
 'table',
 'tbx',
 'textarea',
 'th',
 'title',
 'toast',
 'toggle',
 'tr',
 'txt',
 'updateOption',
 'wrapper'}

In [22]:
page_and_field = defaultdict(set)
for key in value_by_widget.keys():
    if key in {'img', 'icon', 'label', 'star', 'cbx', 'btn', 'btn-NewCandidate', 'tab', 'toggle', 'switch', 'updateOption', 'deleteOption'}:
        for value in value_by_widget[key]:
            print(value)
            list_1 = value.split('/')
            last = list_1.pop()
            string_1 = '/'.join(list_1)
            list_2 = last.split('_')
            first = list_2.pop(0)
            string_2 = '_'.join(list_2)
            page_and_field[(string_1, string_2)].add(key)

Page_Candidates/btn-NewCandidate
Page_TestOverview/switch_TestAccessible_Disable
Page_TestOverview/switch_TestAccessible
Page_Libraries/More_Action/updateOption_relyOnOrder
Page_Libraries/More_Action/updateOption_relyOnName
Page_TestQuestions/cbx_rightAnswer
Page_TestQuestions/cbx_QuestionType
Page_TestQuestions/cbx_Question
Page_TestQuestions/cbx_LibraryQuestion
Page_TestInvited/cbx_Select All
Page_Libraries/More_Action/deleteOption_relyOnName
Page_Libraries/More_Action/deleteOption_relyOnOrder
Page_TestQuestions/label_MultipleChoice
Page_TestOverview/label_StatusOfSetting
Page_TestInvited/btn_Bulk Cancel
Page_SearchTests/btn_delete_dialog_cancel
Page_TakeTest/btn_Submit Test
Page_TakeTest/btn_Load Question
Page_TakeTest/btn_Enter Test
Page_Libraries/More_Action/btn_Cancel
Page_TestQuestions/btn_CancelButton
Page_Create Member/btn_modal_add
Page_TestQuestions/btn_TestQuestion
Page_PreviewTest/btn_ResetAnswer
Page_UpdateProfile/btn_Submit
Page_PreviewTest/btn_Submit
Page_Test Submitted

In [23]:
page_and_field

defaultdict(set,
            {('Page_Candidates', ''): {'btn-NewCandidate'},
             ('Page_TestOverview', 'TestAccessible_Disable'): {'switch'},
             ('Page_TestOverview', 'TestAccessible'): {'switch'},
             ('Page_Libraries/More_Action', 'relyOnOrder'): {'deleteOption',
              'updateOption'},
             ('Page_Libraries/More_Action', 'relyOnName'): {'deleteOption',
              'updateOption'},
             ('Page_TestQuestions', 'rightAnswer'): {'cbx'},
             ('Page_TestQuestions', 'QuestionType'): {'cbx'},
             ('Page_TestQuestions', 'Question'): {'cbx'},
             ('Page_TestQuestions', 'LibraryQuestion'): {'cbx'},
             ('Page_TestInvited', 'Select All'): {'cbx'},
             ('Page_TestQuestions', 'MultipleChoice'): {'label'},
             ('Page_TestOverview', 'StatusOfSetting'): {'label'},
             ('Page_TestInvited', 'Bulk Cancel'): {'btn'},
             ('Page_SearchTests', 'delete_dialog_cancel'): {'btn'},
     

In [ ]:
list_1 = 'Page_Libraries/Question_Dialog/tbx_ChoiceValue'.split('/')
list_1.pop()
'/'.join(list_1)

In [ ]:
'Page_PreviewTest/textarea_Feedback'

In [ ]:
{'page': 'PreviewTest', 'field': 'Feedback', 'widget': 'INPUT'}

# This is what you need (+ func_obj_dict)

In [25]:
parsed_test_cases = [[{'V': 'navigate', 'page': 'login'},
  {'V': 'enter',
   'location': 'to email textbox',
   'value': 'admin1@mail.com',
   'page': 'login'},
  {'V': 'enter',
   'location': 'to password textbox',
   'value': 'Admin@123',
   'page': 'login'},
  {'V': 'click', 'value': 'Login', 'page': 'login'}],
 [{'V': 'go', 'page': 'login'},
  {'V': 'input',
   'location': 'to login',
   'value': 'an invalid credentials',
   'page': 'login'},
  {'V': 'click', 'value': 'on login button', 'page': 'login'}],
 [{'V': 'click', 'value': 'on the top right corner', 'page': 'logout'},
  {'V': 'click', 'value': 'Sign out', 'page': 'logout'}],
 [{'V': 'click', 'value': 'on the top right', 'page': 'change password'},
  {'V': 'click',
   'value': 'on change password link',
   'page': 'change password'},
  {'V': 'require', 'page': 'change password'},
  {'V': 'click', 'value': 'on submit button', 'page': 'change password'}],
 [{'V': 'click',
   'value': 'on the top right corner',
   'page': 'change password'},
  {'V': 'click', 'value': 'Change password', 'page': 'change password'},
  {'V': 'do', 'page': 'change password'},
  {'V': 'click', 'value': 'Submit', 'page': 'change password'}],
 [{'V': 'click', 'value': "'s avatar", 'page': 'Update my personal info'},
  {'V': 'click', 'page': 'Update my personal info'},
  {'V': 'input',
   'location': 'on the update my profile page',
   'value': 'full name',
   'page': 'Update my personal info'},
  {'V': 'click',
   'value': 'on submit button',
   'page': 'Update my personal info'}],
 [{'V': 'click',
   'value': 'on the top right corner',
   'page': 'Update my personal info'},
  {'V': 'click', 'value': 'My Profile', 'page': 'Update my personal info'},
  {'V': 'leave', 'page': 'Update my personal info'},
  {'V': 'enter',
   'location': 'to textbox phone',
   'value': '12345678',
   'page': 'Update my personal info'},
  {'V': 'click', 'value': 'Submit', 'page': 'Update my personal info'}]]

In [ ]:
# import math
# counter = 0
# for idx, (case, script) in enumerate(zip(test_cases_df['Full manual test cases'].tolist(), test_cases_df['test_scripts'].to_list())):
#     print(counter)
#     if counter <= 7:
#         if isinstance(case, float):
#             continue
#         else:
#             print(case)
#             print()
#             print(parsed_test_cases[counter])
#             print()
#             print(script)
#             print()
#             print('-----')
#             print()
#             counter += 1
#     else:
#         break

In [64]:
import Levenshtein

In [65]:
Levenshtein.distance('login', 'Page_Password')

12

In [66]:
widget_dict_1

defaultdict(<function __main__.<lambda>()>,
            {'Page_SearchTests': defaultdict(set,
                         {'txt': {'Page_SearchTests/txt_dropdown_menu',
                           'Page_SearchTests/txt_search',
                           'Page_SearchTests/txt_toggle_option'},
                          'btn': {'Page_SearchTests/btn_delete_dialog_cancel',
                           'Page_SearchTests/btn_delete_dialog_ok'},
                          'nav': {'Page_SearchTests/nav_testStatus'},
                          'cpn': {'Page_SearchTests/cpn_more',
                           'Page_SearchTests/cpn_test_modal',
                           'Page_SearchTests/cpn_tests'},
                          'toggle': {'Page_SearchTests/toggle_more'},
                          'successToast': {'Page_SearchTests/successToast'}}),
             'Page_Set Password': defaultdict(set,
                         {'txt': {'Page_Set Password/txt_Password',
                           'Page_Set Pass

In [26]:
path_dict = defaultdict(lambda: defaultdict(set))
for values in func_obj_dict.values():
    for value in values:
        get_path_list = value.split('/')
        path = '/'.join(get_path_list[0:-1])
        get_element_list = get_path_list[-1].split('_')
        element = get_element_list[1:]
        if len(get_element_list) > 1:
            element = '_'.join(get_element_list[1:])
            path_dict[path][element].add(value)
        else:
            element = get_element_list[0]
            path_dict[path][element].add(value)

In [27]:
path_dict

defaultdict(<function __main__.<lambda>()>,
            {'Page_SearchTests': defaultdict(set,
                         {'search': {'Page_SearchTests/txt_search'},
                          'delete_dialog_cancel': {'Page_SearchTests/btn_delete_dialog_cancel'},
                          'toggle_option': {'Page_SearchTests/txt_toggle_option'},
                          'testStatus': {'Page_SearchTests/nav_testStatus'},
                          'more': {'Page_SearchTests/cpn_more',
                           'Page_SearchTests/toggle_more'},
                          'delete_dialog_ok': {'Page_SearchTests/btn_delete_dialog_ok'},
                          'successToast': {'Page_SearchTests/successToast'},
                          'dropdown_menu': {'Page_SearchTests/txt_dropdown_menu'},
                          'test_modal': {'Page_SearchTests/cpn_test_modal'},
                          'tests': {'Page_SearchTests/cpn_tests'}}),
             'Page_Set Password': defaultdict(set,
          

In [215]:
def location_modifier(location):
    location = ' '.join(location.split('_'))
    all_list = (re.findall('[A-Z]*[a-z]*', location))
    starter = ''
    for string in all_list:
        if string == '':
            continue
        else:
            starter += string + ' ' 
    return starter.rstrip()

In [218]:
(re.findall('[A-Z]*[A-Z]?[a-z]*', 'PWMessage'))

['PWMessage', '']

In [216]:
for key in path_dict.keys():
    for inner_key in path_dict[key].keys():
        print(f'{inner_key}, {location_modifier(inner_key)}')

search, search
delete_dialog_cancel, delete dialog cancel
toggle_option, toggle option
testStatus, test Status
more, more
delete_dialog_ok, delete dialog ok
successToast, success Toast
dropdown_menu, dropdown menu
test_modal, test modal
tests, tests
Password, Password
Repassword, Repassword
Set Password, Set Password
TestTags, Test Tags
TextOrNumberField, Text Or Number Field
Successfully, Successfully
PreviewTest, Preview Test
OptionDropdown, Option Dropdown
TestAccessible, Test Accessible
EditRichText, Edit Rich Text
Ok_Toggle_Accessible, Ok Toggle Accessible
CancelConfirmModal, Cancel Confirm Modal
Option, Option
SwitchSetting, Switch Setting
Publish, Publish
Ok_Publish, Ok Publish
Cancel_Publish, Cancel Publish
ShowingTextOrNumberField, Showing Text Or Number Field
Email_Templates, Email Templates
Overview, Overview
Confirm, Confirm
Save, Save
PublishTest, Publish Test
TestAccessible_Disable, Test Accessible Disable
TestName, Test Name
RichText, Rich Text
Test_Status, Test Status
C

In [102]:
action_widget = func_obj_dict.copy()

In [103]:
action_widget['enter'] = action_widget.pop('setText')

In [104]:
action_widget['input'] = action_widget['enter']

In [81]:
action_widget = {'input': {'tbx', 'inp', 'input', 'textarea', 'ipn', 'ta', 'ckeditor', 'ckedior', 'updateOption', 'deleteOption'},
'enter': {'tbx', 'inp', 'input', 'tbx', 'textarea', 'ipn', 'ta', 'ckeditor', 'ckedior', 'updateOption', 'deleteOption'},
'click': {'img', 'icon', 'lbl', 'label', 'star', 'cbx', 'btn', 'tab', 'toggle', 'switch'},
'click_content': {'radio', 'select', 'slt', 'rdi', 'link'},
'container': {'Span', 'span', 'modal', 'wrapper', 'div', 'dialog', 'nav'},
'readable': {'successToast', 'th', 'title', 'toast', 'txt', 'tr', 'content', 'cpn', 'table'}}

In [29]:
def get_page(page):
    min_dist = np.inf
    min_key = ''
    for key in path_dict.keys():
        distance = Levenshtein.distance(page, key.lower())
        if distance < min_dist:
            min_dist = distance
            min_key = key
    return min_key

In [90]:
# def get_object_path(parsed_test_case):
#     page = get_page(parsed_test_case['page'])
#     if 'location' in parsed_test_case.keys():
#         location = get_location(page, parsed_test_case['location'])
#     else:
#         location = get_location(page, parsed_test_case['value'])
#     for element in path_dict[page][location]:
#         if element.split('/')[-1].split('_')[0] in action_widget[parsed_test_case['V']]:
#             return element
#         else:
#             return None

In [105]:
action_widget

{'click': {'Module_Navigation/btn_ChangePassword',
  'Module_Navigation/btn_Libraries',
  'Module_Navigation/lbl_Avatar',
  'Module_Navigation/nav_Members',
  'Module_Navigation/nav_item_libraries',
  'Module_Navigation/nav_item_tests',
  'Page_AddTest/EmptyPage/btn_CreateTest-Empty',
  'Page_AddTest/btn_CreateTest',
  'Page_Candidates/btn-NewCandidate',
  'Page_Candidates/btn_CancelledCandidate',
  'Page_Candidates/btn_InvitedCandidate',
  'Page_Candidates/btn_ScoredCandidate',
  'Page_Candidates/btn_SearchButton',
  'Page_Candidates/btn_SubmittedCandidate',
  'Page_Candidates/nav_TestName',
  'Page_ChangePassword/btn_Submit',
  'Page_ChangePassword/tbx_CurrentPassword',
  'Page_Create Member/btn_Create Member',
  'Page_Create Member/btn_modal_add',
  'Page_Create Member/select_Role',
  'Page_Create Member/txt_Role',
  'Page_EmailTemplates/btn_Email Type',
  'Page_EmailTemplates/btn_email_Template',
  'Page_EmailTemplates/ipn_Subjective',
  'Page_EmailTemplates/tab_Cancelled_Email',
 

In [126]:
action_widget['wait'] = set()
for key in action_widget.keys():
    if 'wait' in key.lower():
        action_widget['wait'].update((action_widget[key]))

In [135]:
action_widget['get'] = action_widget.pop('getText')

In [129]:
action_widget['wait']

{'Module_Navigation/nav_item_tests',
 'Module_Navigation/txt_title',
 'Page_AddTest/successToast',
 'Page_ChangePassword/txt_ConfirmPWErrorMessage',
 'Page_ChangePassword/txt_CurrentPWErrorMessage',
 'Page_ChangePassword/txt_NewPWErrorMessage',
 'Page_ChangePassword/txt_SuccessMessage',
 'Page_ChangePassword/txt_Title',
 'Page_EmailTemplates/ipn_Subjective',
 'Page_EmailTemplates/tab_Cancelled_Email',
 'Page_EmailTemplates/tab_Invite_Email',
 'Page_EmailTemplates/tab_Reminder_Email',
 'Page_EmailTemplates/tab_Thanks_Email',
 'Page_Libraries/Question_Dialog/dialog_UpdateQuestion',
 'Page_Libraries/Question_Dialog/toast_Sucess',
 'Page_Libraries/Question_Dialog/txt_InvalidSugestedAnswer',
 'Page_Libraries/Question_Dialog/txt_MCQ_ErrorMessage',
 'Page_Libraries/link_homeLib',
 'Page_Libraries/txt_SuccessfullMessage',
 'Page_Login/btn_Login',
 'Page_Login/txt_EmailErrorMessage',
 'Page_Login/txt_PasswordErrorMessage',
 'Page_TakeTest/btn_Enter Test',
 'Page_TakeTest/btn_Load Question',
 'P

In [30]:
!python -m spacy download en_core_web_md 

     |████████████████████████████████| 45.7 MB 1.1 MB/s eta 0:00:01    |████▌                           | 6.4 MB 589 kB/s eta 0:01:07     |██████████████████              | 25.8 MB 445 kB/s eta 0:00:45
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [62]:
import spacy

nlp = spacy.load("en_core_web_md")  # make sure to use larger package!
# doc1 = nlp("Update my personal info")
# doc2 = nlp("Update Library")

# # Similarity of two documents
# print(doc1, "<->", doc2, doc1.similarity(doc2))

In [109]:
def get_location(page, location):
    min_dist = np.inf
    min_key = ''
    for key in path_dict[page].keys():
        distance = Levenshtein.distance(location, key.lower())
        if distance < min_dist:
            min_dist = distance
            min_key = key
    return min_key

In [163]:
def page_str_modifier(page):
    if len(page.split('/')) > 1:
        page = ''.join(page.split('/')[1:])
        page = ''.join(page.split('_'))
    else:
        page = ''.join(page.split('_'))[1:]
        print(page)
    page = ' '.join(re.findall('[A-Z][a-z]+', page))
    return page

In [164]:
page_str_modifier('delete_dialog_ok')

eletedialogok


''

In [111]:
def get_page(page):
    max_sim = -np.inf
    max_key = ''
    for key in path_dict.keys():
        similarity = nlp(page).similarity(nlp(page_str_modifier(key)))
        if similarity > max_sim:
            max_sim = similarity
            max_key = key
    return max_key

In [110]:
def get_object_path(parsed_test_case):
    page = get_page(parsed_test_case['page'])
    if 'location' in parsed_test_case.keys():
        location = get_location(page, parsed_test_case['location'])
    else:
        location = get_location(page, parsed_test_case['value'])
    for element in path_dict[page][location]:
        if element in action_widget[parsed_test_case['V']]:
            return element
        else:
            return None

In [114]:
test_cases = [[{'V': 'enter',
   'location': 'to email textbox',
   'value': 'admin1@mail.com',
   'page': 'login'},
  {'V': 'enter',
   'location': 'to password textbox',
   'value': 'Admin@123',
   'page': 'login'},
  {'V': 'click', 'value': 'Login', 'page': 'login'},
  {'V': 'wait',
   'value': 'title to be present for seconds',
   'time': 30,
   'page': 'login'}],
 [{'V': 'enter',
   'location': 'to email textbox',
   'value': 'invalid@wrong',
   'page': 'login'},
  {'V': 'enter',
   'location': 'to password textbox',
   'value': 'invalidpassword',
   'page': 'login'},
  {'V': 'click', 'value': 'Login', 'page': 'login'},
  {'V': 'wait',
   'value': 'email error message to appear for seconds',
   'time': 3,
   'page': 'login'},
  {'V': 'get', 'value': 'email error message', 'page': 'login'},
  {'V': 'check', 'page': 'login'},
  {'V': 'wait',
   'value': 'password error message to appear for seconds',
   'time': 3,
   'page': 'login'},
  {'V': 'get', 'value': 'password error message', 'page': 'login'},
  {'V': 'check', 'page': 'login'}],
 [{'V': 'click', 'value': 'the avatar', 'page': 'logout'},
  {'V': 'click', 'value': 'Sign out', 'page': 'logout'},
  {'V': 'wait', 'value': 'Login', 'time': 10, 'page': 'logout'}],
 [{'V': 'click', 'value': "user 's avatar", 'page': 'change password'},
  {'V': 'click', 'value': 'change password link', 'page': 'change password'},
  {'V': 'wait',
   'value': 'title to appear for s',
   'time': 10,
   'page': 'change password'},
  {'V': 'enter', 'value': 'Admin@123', 'page': 'change password'},
  {'V': 'enter', 'value': 'Admin@1234', 'page': 'change password'},
  {'V': 'enter',
   'location': 'to confirm password',
   'value': 'Admin@1234',
   'page': 'change password'},
  {'V': 'click', 'value': 'submit', 'page': 'change password'}],
 [{'V': 'click', 'value': 'the avatar', 'page': 'change password'},
  {'V': 'click', 'value': 'Change password', 'page': 'change password'},
  {'V': 'wait',
   'value': 'title to appear for s',
   'time': 10,
   'page': 'change password'},
  {'V': 'enter', 'value': 'current password "', 'page': 'change password'},
  {'V': 'enter', 'value': 'new password', 'page': 'change password'},
  {'V': 'enter',
   'location': 'to confirm password "',
   'value': 'new password',
   'page': 'change password'},
  {'V': 'click', 'value': 'submit', 'page': 'change password'}],
 [{'V': 'click',
   'value': 'my profile link',
   'page': 'Update my personal info'},
  {'V': 'wait',
   'value': 'to appear for s',
   'time': 20,
   'page': 'Update my personal info'},
  {'V': 'enter', 'value': 'new name', 'page': 'Update my personal info'},
  {'V': 'enter', 'value': '1234567890', 'page': 'Update my personal info'},
  {'V': 'click', 'value': 'submit', 'page': 'Update my personal info'}],
 [{'V': 'click', 'value': 'the avatar', 'page': 'Update my personal info'},
  {'V': 'click', 'value': 'My Profile', 'page': 'Update my personal info'},
  {'V': 'wait',
   'value': 'for email to appear for s',
   'time': 20,
   'page': 'Update my personal info'},
  {'V': 'enter', 'value': 'new full name', 'page': 'Update my personal info'},
  {'V': 'enter', 'value': '12345678', 'page': 'Update my personal info'},
  {'V': 'click', 'value': 'submit', 'page': 'Update my personal info'}]]

In [115]:
get_object_path({'V': 'enter',
   'location': 'to password textbox',
   'value': 'Admin@123',
   'page': 'login'})

'Page_Login/tbx_Password'

In [116]:
get_object_path({'V': 'click', 'value': 'Login', 'page': 'login'})

'Page_Login/btn_Login'

In [128]:
get_object_path({'V': 'wait',
   'value': 'title to be present for seconds',
   'time': 30,
   'page': 'login'})

'Page_Login/txt_EmailErrorMessage'

In [130]:
get_object_path({'V': 'enter',
   'location': 'to email textbox',
   'value': 'invalid@wrong',
   'page': 'login'})

'Page_Login/tbx_Email'

In [131]:
get_object_path({'V': 'enter',
   'location': 'to password textbox',
   'value': 'invalidpassword',
   'page': 'login'})

'Page_Login/tbx_Password'

In [132]:
get_object_path({'V': 'click', 'value': 'Login', 'page': 'login'})

'Page_Login/btn_Login'

In [133]:
get_object_path({'V': 'wait',
   'value': 'email error message to appear for seconds',
   'time': 3,
   'page': 'login'})

'Page_Login/txt_EmailErrorMessage'

In [136]:
get_object_path({'V': 'get', 'value': 'email error message', 'page': 'login'})

'Page_Login/txt_EmailErrorMessage'

In [138]:
get_object_path({'V': 'wait',
   'value': 'password error message to appear for seconds',
   'time': 3,
   'page': 'login'})

'Page_Login/txt_PasswordErrorMessage'

In [139]:
get_object_path({'V': 'get', 'value': 'password error message', 'page': 'login'})

'Page_Login/txt_PasswordErrorMessage'

In [140]:
get_object_path({'V': 'click', 'value': 'the avatar', 'page': 'logout'})

In [141]:
get_object_path({'V': 'click', 'value': 'Sign out', 'page': 'logout'})

'Page_Login/btn_Logout'

In [142]:
get_object_path({'V': 'wait', 'value': 'Login', 'time': 10, 'page': 'logout'})

'Page_Login/btn_Login'

In [143]:
get_object_path({'V': 'click', 'value': "user 's avatar", 'page': 'change password'})

In [144]:
get_object_path({'V': 'click', 'value': 'change password link', 'page': 'change password'})

In [145]:
get_object_path({'V': 'wait',
   'value': 'title to appear for s',
   'time': 10,
   'page': 'change password'})

In [146]:
get_page('change password')

'Page_ChangePassword'

In [149]:
path_dict['Page_ChangePassword']

defaultdict(set,
            {'NewPassword': {'Page_ChangePassword/tbx_NewPassword'},
             'ConfirmPassword': {'Page_ChangePassword/tbx_ConfirmPassword'},
             'CurrentPassword': {'Page_ChangePassword/tbx_CurrentPassword'},
             'Submit': {'Page_ChangePassword/btn_Submit'},
             'ConfirmPWErrorMessage': {'Page_ChangePassword/txt_ConfirmPWErrorMessage'},
             'NewPWErrorMessage': {'Page_ChangePassword/txt_NewPWErrorMessage'},
             'CurrentPWErrorMessage': {'Page_ChangePassword/txt_CurrentPWErrorMessage'},
             'Title': {'Page_ChangePassword/txt_Title'},
             'SuccessMessage': {'Page_ChangePassword/txt_SuccessMessage'}})

In [150]:
def get_location(page, location):
    min_dist = np.inf
    min_key = ''
    for key in path_dict[page].keys():
        distance = Levenshtein.distance(location, key.lower())
        print(f'{key}: {distance}')
        if distance < min_dist:
            min_dist = distance
            min_key = key
    return min_key

In [151]:
get_location('Page_ChangePassword', 'title to appear for s')

NewPassword: 16
ConfirmPassword: 17
CurrentPassword: 15
Submit: 20
ConfirmPWErrorMessage: 20
NewPWErrorMessage: 19
CurrentPWErrorMessage: 18
Title: 16
SuccessMessage: 19


'CurrentPassword'

In [152]:
nlp('title to appear for s').similarity(nlp('title'))

0.6815145029388178

In [159]:
nlp('title to appear for s').similarity(nlp('ConfirmPWErrorMessage'))

<ipython-input-159-982dde436c65>:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  nlp('title to appear for s').similarity(nlp('ConfirmPWErrorMessage'))


0.0